In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
from PIL import Image
import requests
from io import BytesIO


In [2]:
def get_(keyword, category_id):
    q = f'''
    select id, name, keyword, 
        case
            when name like '%{keyword}%' or keyword like '%{keyword}%' then 1
            else 0
        end as label
    from service1_quicket.product_info
    where category_id = {category_id} and status IN (0, 1)
    '''
    return pd.read_sql(q, con=bun_dw)

In [3]:
golf = get_('골프채', 700600300)

In [36]:
print(golf.shape[0])
59982/ golf.shape[0]

87935


0.6821174731335645

In [4]:
golf.groupby('label').count()

,id,name,keyword
label,,,
0,43316,43315,27756
1,54964,54964,54115


In [5]:
golf[golf['label'] == 0]

,id,name,keyword,label
0,99537024,푸마골프 클래식 레더 골프벨트,"골프벨트,벨트,푸마벨트,PUMA벨트,푸마골프",0
1,99590976,힐크릭 HCCB-7001 캐디백,None,0
4,100007555,아다바트 보스톤백,None,0
5,100272513,풋조이 DNA Helix 헬릭스 보아 53326 골프화 2018,"골프,골프용품,골프화,FootJoy,풋조이",0
9,100265986,풋조이 드라이 조이 투어 남성 골프화 53668,"골프,골프용품,골프화,FootJoy,풋조이",0
...,...,...,...,...
98275,147841353,테일러메이드 ATV KBS웨지플랙스52도웨지 (US스펙),None,0
98276,147927719,오토파워 TX5885 5S 5X 5XX 6X 6XX 샤프트 신품급,"골프,클럽기타,샤프트,드라이버 샤프트",0
98277,147928782,테일러메이드M4드라이버,None,0
98278,147928849,핑G25 7번 21도 아타스 75S 우드 팝니다,"핑,핑G25우드,핑우드,ping",0


In [46]:
golf.to_csv('golf.csv', index=False)

In [6]:
import re
def preprocess_product_name(text):
    text = str(text)
    text = re.sub(r"[^가-힣ㄱ-ㅎㅏ-ㅣA-z0-9-:/!@#$%^?&*()_=+.~{}\[\]]|\\", " ", text)
    text = re.sub(r'\s+', ' ', text)
    return text.lower().strip()

In [7]:
golf['name'] = golf['name'].map(preprocess_product_name)
golf['keyword'] = golf['keyword'].map(preprocess_product_name)

In [8]:
golf.drop_duplicates('name', inplace=True)

In [9]:
names = golf['name'].values.tolist()

In [10]:
train_set = golf['name'].map(lambda x: str(x) + ' ').values + golf['keyword'].values

In [17]:
product_name_list = 'products_for_pretrain.txt'
with open(product_name_list, 'w') as file:
    file.writelines(map(lambda x: str(x) + '\n', train_set))

In [21]:
sent2vec_model_file = 'sent2vec_model.bin'
snet2vec_index_file = 'sent2vec_index.bin'
sent2vec_lib_path = './sent2vec'

In [20]:
import os

param = {
    "input": product_name_list,
    "output": sent2vec_model_file.split('.')[0],
    "minCount": 3,
    "dim": 100,
    "epoch": 10,
    "lr": 0.2,
    "wordNgrams": 5,
    "loss": "ns",
    "neg": 10,
    "thread": os.cpu_count(),
    "t": "0.000005",
    "dropoutK": 4,
    "minCountLabel": 20,
    "bucket": 500000,
    "maxVocabSize": 100000,
    "numCheckPoints": 0,
    "verbose": 1
}
parse_param = " ".join(["-{} {}".format(k, v) for k, v in param.items()])

In [26]:
cmd = f"{sent2vec_lib_path}/fasttext sent2vec "
cmd = cmd + parse_param
print(cmd)
os.system(cmd)

./sent2vec/fasttext sent2vec -input products_for_pretrain.txt -output sent2vec_model -minCount 3 -dim 100 -epoch 10 -lr 0.2 -wordNgrams 5 -loss ns -neg 10 -thread 16 -t 0.000005 -dropoutK 4 -minCountLabel 20 -bucket 500000 -maxVocabSize 100000 -numCheckPoints 0 -verbose 1


0

In [47]:
import sent2vec
model = sent2vec.Sent2vecModel()
model.load_model(sent2vec_model_file)

AttributeError: module 'sent2vec' has no attribute 'Sent2vecModel'

In [38]:
q = '''
SELECT id,
       name,
       keyword
FROM service1_quicket.product_info
WHERE category_id = 700600400
  AND (name LIKE '%여성%'
       OR name LIKE '%여자%'
       OR keyword LIKE '%여성%'
       OR keyword LIKE '%여자%')
  AND status IN (0,
                 1)
'''
golf_female = pd.read_sql(q, con=bun_dw)

In [39]:
golf_female

,id,name,keyword
0,100443777,와이드앵글 여성 티,"와이드앵글,여성티"
1,100057859,파리게이츠 바지,"파리게이츠,파리게이츠여성,파리게이츠여성바지,골프복,여성골프복"
2,101606528,여성 골프웨어 (6종류),"갤러웨이,헤지스,파리게이츠,아다바트,골프웨어"
3,99962568,FILA GOLF 볼챙여성모자,필라골프모자
4,100191306,(100)여성 그렉노먼 골프웨어 카라 티셔츠,100사이즈
...,...,...,...
18264,147934196,여성골프웨어,"pxg,pxg바람막이,골프바람막이,골프웨어,골프여성복"
18265,147934292,여성골프웨어,"pxg,pxg스커트,pxg치마,골프스커트,골프치마"
18266,147934467,[여성골프] 타이틀 체크세미팬츠,"여성타이틀바지,여성타이틀팬츠,타이틀여성팬츠,타이틀여성바지,타이틀팬츠"
18267,147935445,무료배송 타이틀 리스트 신자크 팬츠,"타이틀,타이틀리스트,타이틀하의,여자골프,골프"


In [40]:
q = '''
SELECT id,
       name,
       keyword
FROM service1_quicket.product_info
WHERE category_id = 700600400
  AND (name LIKE '%남성%'
       OR name LIKE '%남자%'
       OR keyword LIKE '%남성%'
       OR keyword LIKE '%남자%')
  AND status IN (0,
                 1)
'''
golf_male = pd.read_sql(q, con=bun_dw)

In [41]:
golf_male

,id,name,keyword
0,99695233,남성용 골프화.파리게이츠 골프화 270사이즈 정품 새상품 ㅡ착불 12만,None
1,99726785,나이키골프 남성카라셔츠,"나이키골프,남성카라티,골프웨어,골프티셔츠"
2,99682632,♡까스텔바작남성골프바지,None
3,100466769,🚫🚫세일🛇🛇타이? 남자 벌집바지,"타이틀리스트,남자바지,골프남자바지,골프의류,골프바지"
4,100060755,나이키34남성골프바지/남성골프웨어/스판/퐢로또/m,None
...,...,...,...
12146,147940191,새상품 남성33,남성바지
12147,147940498,"잭니클라우스 남자골프바지 32,33","잭니클라우스,32,33,골프바지"
12148,147942012,아디다스 남성28 에서30,트레이닝복
12149,147934749,남성 골프웨어,"PXG,PXG남성상의,PXG이너티,PXG골프,골프"


In [42]:
pd.merge(golf_female, golf_male, on='id')

,id,name_x,keyword_x,name_y,keyword_y
0,103956309,골프양말 5켤레 (여성용/남성용),"골프양말,골프화,스포츠양말,양말,골프",골프양말 5켤레 (여성용/남성용),"골프양말,골프화,스포츠양말,양말,골프"
1,105844731,타이틀리스트 남여공용 남자95/S 여자77/L,"타이틀리스트,골프복,골프웨어,여름골프,여름골프티",타이틀리스트 남여공용 남자95/S 여자77/L,"타이틀리스트,골프복,골프웨어,여름골프,여름골프티"
2,109393613,❤무료배송❤신상 남성.여성 공용 골프벨트,반팔티,❤무료배송❤신상 남성.여성 공용 골프벨트,반팔티
3,107472380,새제품) 가격인하***프리미엄 올양피.골프장갑.여성/남성 골프장갑.새제품,"골프장갑,골프",새제품) 가격인하***프리미엄 올양피.골프장갑.여성/남성 골프장갑.새제품,"골프장갑,골프"
4,109464531,파리게이츠(정품)골프바지(상태최상)남녀공용,"패리게이츠,골프웨어,여성골프바지,파리게이츠정품바지,남성골프바지",파리게이츠(정품)골프바지(상태최상)남녀공용,"패리게이츠,골프웨어,여성골프바지,파리게이츠정품바지,남성골프바지"
...,...,...,...,...,...
457,147656300,남성여성골프웨어,"pxg,pxg골프모자,골프웨어,골프모자,골프캡",남성여성골프웨어,"pxg,pxg골프모자,골프웨어,골프모자,골프캡"
458,146819029,남성골프웨어,"여성골프웨어,여성골프복,여자골프웨어,여자골프복,골프웨어",남성골프웨어,"여성골프웨어,여성골프복,여자골프웨어,여자골프복,골프웨어"
459,147932471,남성 골프웨어,"PXG,PXG여자,PXG남자,PXG남자골프,골프",남성 골프웨어,"PXG,PXG여자,PXG남자,PXG남자골프,골프"
460,146391927,골프장갑,"골프,골프모자,골프장갑,남자장갑,여자장갑",골프장갑,"골프,골프모자,골프장갑,남자장갑,여자장갑"


In [49]:
q = '''
    SELECT id,
           name,
           keyword,
           CASE
               WHEN name LIKE '%남성%'
                    OR keyword LIKE '%남성%' THEN 1
               WHEN name LIKE '%여성%'
                    OR keyword LIKE '%여성%' THEN 2
               ELSE 0
           END AS label
    FROM service1_quicket.product_info
    WHERE category_id = 700600400
      AND status IN (0, 1)
'''

df = pd.read_sql(q, con=bun_dw)


def preprocess_product_name(text):
    text = str(text)
    text = re.sub(r"[^가-힣ㄱ-ㅎㅏ-ㅣA-z0-9-:/!@#$%^?&*()_=+.~{}\[\]]|\\", " ", text)
    text = re.sub(r'\s+', ' ', text)
    return text.lower().strip()

In [50]:
df.groupby('label').size()

label
0    42385
1    11023
2    16714
dtype: int64

In [53]:
df['name'] = df['name'].map(preprocess_product_name)
df['keyword'] = df['keyword'].map(preprocess_product_name)

In [55]:
df.to_csv('golf_wear.csv', index=False)